# Segmenting ROI and RONI in CT scan
### Author- Anupreet Singh(UMBC Campus ID-UK43298)

In [2]:
## Setting up dependecies in the environment 
# %pip install opencv-python matplotlib 

!pip install opencv-python
!pip install numpy
!pip install matplotlib
!pip install bchlib
!pip install bitarray
!pip install scikit-image

import cv2
import numpy as np
import matplotlib.pyplot as plt

import os
import hashlib
import bchlib
import binascii
from bitarray import bitarray
from skimage.metrics import structural_similarity

In [ ]:
def watermark_image(input_path):
    #For some reason the globally imported dependencies weren't available in this function so we use the global keyword to tell the compiler we are referring to them
    global cv2, np, plt, os, hashlib, bchlib, binascii, bitarray, structural_similarity

    # Read the image in grayscale mode and store it as a NumPy array(where each element represents the pixel intensity stored as an 8 bit unsigned integer ranging from 0-255) in input_image
    input_image = cv2.imread(input_path, cv2.IMREAD_GRAYSCALE)

    # Check if image was successfully loaded
    if input_image is None:
        raise ValueError("Image not found or path is incorrect.")

    ## Step 2 of 7: Drawing a black Boundary on the input image, in this case the black boundary is just at the edge of the of image
    # Make a copy of the input image so we don't overwrite the original
    image_boundary = input_image.copy()

    # Drawing a black border (1 pixel wide) on all sides of the image by putting the intensity to 0(Black)
    image_boundary[0, :] = 0             # Top row
    image_boundary[-1, :] = 0            # Bottom row
    image_boundary[:, 0] = 0             # Left most column
    image_boundary[:, -1] = 0            # Right most column

   

    ## Step 3 of 7: Use OpenCV's built-in Otsu's method to compute threshold & Step 4 of 7: Turn all pixels greater than T_final white to get binarized_image
    # cv2.threshold returns both threshold and the binarized image
    # All pixels > Tfinal → 255 (white), else → 0 (black)
    T_final, binarized_image  = cv2.threshold(
        image_boundary,      # Input image (must be grayscale)
        0,             # Initial threshold value (ignored when using Otsu)
        255,           # Max value to use in binary thresholding
        cv2.THRESH_BINARY + cv2.THRESH_OTSU #Tells opencv to perform binary thresholding with otsu's method
    )

    ## Step 5 of 7: Find the seed pixels
    # The lungs are dark, and were kept black (0) after binarization.
    #In each of the 4 Quadrants of the image, We're looking for the first black pixel — that’s your seed for that quadrant.

    height, width = binarized_image.shape #Storing the height and width 
    margin = 1 # Setting margin of 1 since edges are the black boundary itself(1st pixel), so we start from pixel on the inside edge of the black boundary

    # Lines dividing the four quadrants
    mid_row = height // 2
    mid_col = width // 2

    seeds = {} # making a dictionary to store the seeds for each quadrant

    # Quadrant 1: Upper-left (start from left edge of center row of Quadrant 1, move right)
    row_q1 = height // 4 
    for x in range(margin, mid_col):
        if binarized_image[row_q1, x] == 0:# The first black pixel we encounter
            seeds['upper_left'] = (row_q1, x)# storing the seed
            print("Upper-left seed:", seeds['upper_left'])# Printing the seed
            break

    # Quadrant 2: Lower-left (start from left edge of center row of Quadrant 2, move right)
    row_q2 = 3 * height // 4
    for x in range(margin, mid_col):
        if binarized_image[row_q2, x] == 0:
            seeds['lower_left'] = (row_q2, x)# storing the seed
            print("Lower-left seed:", seeds['lower_left'])# Printing the seed
            break

    # Quadrant 3: Upper-right (start from right edge of center row of Quadrant 3, move left)
    row_q3 = height // 4
    for x in range(width - 1 - margin, mid_col, -1):
        if binarized_image[row_q3, x] == 0:# The first black pixel we encounter
            seeds['upper_right'] = (row_q3, x)# storing the seed
            print("Upper-right seed:", seeds['upper_right'])# Printing the seed
            break

    # Quadrant 4: Lower-right (start from right edge of center row of Quadrant 4, move left)
    row_q4 = 3 * height // 4
    for x in range(width - 1 - margin, mid_col, -1):
        if binarized_image[row_q4, x] == 0:# The first black pixel we encounter
            seeds['lower_right'] = (row_q4, x)# storing the seed
            print("Lower-right seed:", seeds['lower_right'])# Printing the seed
            break


    ## Printing the 4-Quadrant Image with seeds
    import cv2
    import matplotlib.pyplot as plt

    # Assuming binarized_image and seeds dictionary already exist

    # Convert image to RGB so we can draw colored markers
    image_rgb = cv2.cvtColor(binarized_image * 255, cv2.COLOR_GRAY2RGB)

    # Draw a red dot at each seed
    for key, (row, col) in seeds.items():
        cv2.circle(image_rgb, (col, row), radius=5, color=(255, 0, 0), thickness=-1)

    # Draw lines to split into 4 quadrants
    height, width = binarized_image.shape
    mid_row = height // 2
    mid_col = width // 2
    cv2.line(image_rgb, (0, mid_row), (width, mid_row), color=(0, 255, 0), thickness=1)
    cv2.line(image_rgb, (mid_col, 0), (mid_col, height), color=(0, 255, 0), thickness=1)

    ## Step 6: Creating a tagged image
    tagged_image = np.where(binarized_image == 255, 1, 0).astype(np.uint8)

    ## Step 7 of 7: Growing region from Seeds
    # Basically starts at a seed and expands to make the successive neighbours black, which were black in the tagged_image

    # Make a white copy of the image to draw the lungs on basically
    result_image = np.ones_like(binarized_image, dtype=np.uint8) * 255

    # Mask to keep track of visited pixels
    visited = np.zeros_like(tagged_image, dtype=bool)

    # 4-connected neighbor directions(above, below, left, right)
    neighbors = [(-1, 0), (1, 0), (0, -1), (0, 1)]

    from collections import deque # importing deque for BFS basically 

    # Loop through all 4 quadrant seeds
    for seed_name in seeds:  # seeds is a dict with 4 quadrant seeds
        seed = seeds[seed_name]#seeds stores the location of the seeds for that quadrant
        if not visited[seed]:  # In case overlapping regions exist, we check if the current seeds has been visited in some other quadrants region expansion
            queue = deque([seed])# initialize the double ended queue with the seed
            visited[seed] = True # Mark the seed as True in the visited mask

            while queue:# Goes until the queue is empty
                y, x = queue.popleft()# retrieves the element from the begining of the queue
                result_image[y, x] = 0# Makes the element black

                for dy, dx in neighbors:#Iterates over 4 neighbours of the currently popped element
                    ny, nx = y + dy, x + dx# Location of current neighbour 
                    if (0 <= ny < result_image.shape[0]) and (0 <= nx < result_image.shape[1]): #Just a bound check to make sure the current location is inside the picture
                        if not visited[ny, nx] and tagged_image[ny, nx] == 0:# Checks that this neighbour/pixel hasn't been visited and it was black in the tagged/binarized image
                            queue.append((ny, nx))# Adds neighbour to the queue for its successive neighbour checks and making it black in the result_image
                            visited[ny, nx] = True# sets the neighbour as True in the visited mask


    ## Step 1
    # Setting LSBs of all pixels to zero
    lsb_zeroed_image = input_image.copy()  # creating a copy of the input_image loaded in the beginning
    lsb_zeroed_image = lsb_zeroed_image & 0b11111110  # Mask LSB to 0,  & 0b11111110 performs Bitwise AND operator between each element of the numpy array and 0b11111110, essentially setting last digit of each element to zero

    #Flattening the image into a byte stream sutiable for hashing
    flattened_bytes = lsb_zeroed_image.flatten().tobytes()#.flatten converts the 2D array to 1D array, /tobytes() converts all elements from int to byte form(1 byte can store upto 8 bits(255) info)

    #Computing SHA-512 hash as 128 character hexadecimal sequence(SHA-512 hash gives 512-bit = 128(512/4) hex characters)
    hash_hex = hashlib.sha512(flattened_bytes).hexdigest()  # hexdigest gives the output a hexadecimal representation

    #Converting the hex sequence to ASCII value in binary to get the watermark

    binary_representation = ''.join(f'{ord(char):08b}' for char in hash_hex)  

    #Printing the output
    print(f"SHA-512 Hash Output(Hexadecimal): {hash_hex}")
    print(f"Binary Representation of SHA-512 Hash: {binary_representation}")
    print(f"The length of the binary representation is: {len(binary_representation)}")

    ## Step 2: Srambling the identified RONI pixels in the original Image

    # Define the key (seed value) for reproducibility
    key = 42  # You can change this value to any integer, Ideally even this should be made using a PRNG

    # Basically we are fixing the seed used by np.random() function as "key"
    np.random.seed(key)

    # Making a copy of the original image to scramble
    scrambled_image = input_image.copy()

    # result_image is the final ROI and RONI separated image. 
    # Extracting coordinates of RONI pixels (value = 255)
    roni_coords = np.column_stack(np.where(result_image == 255)) # 2D array where each row represents a coordinate or pixel value=255

    # Extract the pixel values at those RONI positions in input_image
    roni_values = input_image[roni_coords[:, 0], roni_coords[:, 1]]


    # Shuffles the order of intensity values of RONI pixels stored in roni_values in place
    np.random.shuffle(roni_values)

    # Reassign shuffled values back to all RONI positions
    scrambled_image[roni_coords[:, 0], roni_coords[:, 1]] = roni_values

    print(f"Number of RONI pixels: {len(roni_coords)}")
   
    ## Step 3: Embed Watermark in Scrambled RONI

    # In case BCH does not work out, we consider the binary hash as the watermark to be embedded
    bin_watermark = binary_representation

    # Ensure the watermark to be embedded is 1024 bits (from BCH encoding or direct hash)
    assert len(bin_watermark) == 1024, "Error: Watermark to be embedded must be 256 bits."

    # Convert the watermark binary string to a list of integers (0s and 1s)
    watermark_bits = [int(bit) for bit in bin_watermark]

    # Check the number of total available RONI pixels
    num_roni_pixels = len(roni_coords)

    # Sanity check: Ensure there's at least one RONI pixel
    if num_roni_pixels == 0:
        raise ValueError("No RONI pixels available for embedding.")

    WM_coords = roni_coords[:1024]# Selecting the first 1024 rows of coordinates from the RONI coordinates list to embed the watermark into

    # Embed the watermark bits in the LSBs of the selected RONI pixels of scrambled_image
    for idx, bit in enumerate(watermark_bits):
        roni_x, roni_y = WM_coords[idx]# Each row of WM_coords would give us x and y coordinate of the RONI pixel we want to embed the watermark into 
        pixel_value = scrambled_image[roni_x, roni_y]# Retrieve the pixel value of the Watermark Coordinate

        # Modify the LSB
        new_pixel_value = (pixel_value | 1) if bit == 1 else (pixel_value & 0xFE)# If the bit to be embedded is 1, 

        scrambled_image[roni_x, roni_y] = new_pixel_value

    ## Step 4: Unscramble the pixels in RONI to take them back to original position, and display final Watermarked image

    # The key used is the same but we need to Re-seed to the RNG of Numpy to the bring it to the same state as it was when shuffling/scrambling the roni_coords
    np.random.seed(key)

    # Making Index array for helping with getting original order back
    original_indices = np.arange(num_roni_pixels) #np.arange() give a numpy array with values ranging form 0 to num_roni_pixels-1
    shuffled_indices = original_indices.copy() #making copy to shuffling 
    np.random.shuffle(shuffled_indices) # Shuffled

    # Computing inverse permutation using argsort
    reverse_indices = np.argsort(shuffled_indices)# np.argsort() gives an array of indices corresponding to values of shuffled_indices that would result in an ascending/sorted order

    # At this point RONI coordinates in scrambled_image has their values shuffled and then watermark embedded in the first 256 elements in the roni_coordinates list 
    # Extracting RONI values from the scrambled image
    embedded_roni_values = scrambled_image[roni_coords[:, 0], roni_coords[:, 1]]

    # Restoring order of values roni_values as it were before shuffling
    descrambled_values = embedded_roni_values[reverse_indices]

    # Assigning descrambled values to RONI coordinates in the image 
    scrambled_image[roni_coords[:, 0], roni_coords[:, 1]] = descrambled_values

    ## Sanity Check of Structural Similarity to see that the watermark has been embedded
    ssim = structural_similarity(input_image, scrambled_image, data_range=scrambled_image.max() - scrambled_image.min())
    print("Structural Similarity Index", ssim)

    return scrambled_image

# ===SCRIPT CONFIGURATION=====
input_folder="datasets/ctscan/raw/non-COVID"
output_folder="datasets/ctscan/RONI Watermarked/non-COVID"

# Process all images
for filename in os.listdir(input_folder): #go through all files in input_folder
    if filename.lower().endswith((".png")):#If the file name ends with .png
        input_path = os.path.join(input_folder, filename) # Appropriate input_path for the image
        output_path = os.path.join(output_folder, filename)# Appropriate output_path for the watermarked image

        
        try:
            output_image = watermark_image(input_path)
            cv2.imwrite(output_path, output_image)
            print(f"Processed: {filename} → {output_path}")
        except ValueError as e:
            print(e)

        



Upper-left seed: (36, 49)
Lower-left seed: (108, 49)
Upper-right seed: (36, 186)
Lower-right seed: (108, 186)
SHA-512 Hash Output(Hexadecimal): 6a9119505bc647224f73b0ba84c1a36eae2f687abfe266488a60127c77407fd95a2e091ef99139e72e97eb6165943f5b5aa77171f000a38c028f42a34d01e30b
Binary Representation of SHA-512 Hash: 0011011001100001001110010011000100110001001110010011010100110000001101010110001001100011001101100011010000110111001100100011001000110100011001100011011100110011011000100011000001100010011000010011100000110100011000110011000101100001001100110011011001100101011000010110010100110010011001100011011000111000001101110110000101100010011001100110010100110010001101100011011000110100001110000011100001100001001101100011000000110001001100100011011101100011001101110011011100110100001100000011011101100110011001000011100100110101011000010011001001100101001100000011100100110001011001010110011000111001001110010011000100110011001110010110010100110111001100100110010100111001001101110110010101100010